In [ ]:
import pandas as pd
import json
import os

# Function to load JSON data
def load_json_data(file_path):
    """Load JSON data from the given file path."""
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data



In [ ]:
# Function to integrate data from different sources
def integrate_data(schedule_data, call_data, report_data, workers_data):
    report_df = pd.DataFrame(report_data)
    workers_df = pd.DataFrame.from_dict(workers_data, orient='index').reset_index()
    workers_df.columns = ['worker_id', 'name', 'office']

    call_records = []
    for location, calls in call_data.items():
        for call_id, details in calls.items():
            call_records.append({'call_id': call_id, 'location': location, **details})
    call_df = pd.DataFrame(call_records)

    merged_df = call_df.merge(report_df, on='call_id', how='left')
    merged_df = merged_df.merge(workers_df, on='worker_id', how='left')
    return merged_df


In [ ]:
# Function to create additional features for analysis
def create_features(data):
    data['profit_per_minute'] = data['call_profit'] / data['call_time']
    data['call_efficiency'] = data['likely_to_recommend'] * data['profit_per_minute']
    return data


In [ ]:
# Function to update the next week's call schedule for higher profitability
def update_next_weeks_schedule(base_schedule_path, data, output_path, max_calls_per_worker=5):
    base_schedule = load_json_data(base_schedule_path)
    data_sorted = data.sort_values(by=['call_efficiency', 'call_profit'], ascending=False)
    worker_call_count = {}
    next_week_schedule = {}

    for location, calls in base_schedule.items():
        next_week_schedule[location] = {}
        for call_id, details in calls.items():
            for _, row in data_sorted.iterrows():
                worker_id = row['worker_id']
                if worker_id not in worker_call_count:
                    worker_call_count[worker_id] = 0
                if worker_call_count[worker_id] < max_calls_per_worker:
                    if row['call_id'] == call_id:
                        next_week_schedule[location][call_id] = {
                            "date": details["date"],
                            "tlf_number": details["tlf_number"],
                            "technical_problem": row['technical_problem'],
                            "difficulty": row['difficulty'],
                            "commission": row['commission']
                        }
                        worker_call_count[worker_id] += 1
                        break
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w') as file:
        json.dump(next_week_schedule, file, indent=4)
    print(f"Updated profitable schedule saved to {output_path}")



In [ ]:
# Load all required files
schedule_data = load_json_data('call_shedule_0.json')
call_data = load_json_data('calls_0.json')
report_data = load_json_data('call_report_0.json')
workers_data = load_json_data('workers.json')
next_weeks_base_path = 'calls_11.json'

# Integrate data
integrated_df = integrate_data(schedule_data, call_data, report_data, workers_data)

# Create features
enriched_df = create_features(integrated_df)

# Update next week's schedule for profitability
update_next_weeks_schedule(next_weeks_base_path, enriched_df, 'output/updated_next_week_calls.json')
